In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HF_API_KEY"] = os.getenv("HF_API_KEY")
llm = ChatGroq(groq_api_key=groq_api_key, model="llama3-8b-8192")

In [2]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Users\sodjs\Langchain\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
import bs4

loader = WebBaseLoader(
    web_paths = ("https://laurelnose.github.io/tutorial/",
                 "https://laurelnose.github.io/prologue/",
                 "https://laurelnose.github.io/chapter-1/",
                 "https://laurelnose.github.io/chapter-2-iorveth/",
                 "https://laurelnose.github.io/chapter-2-roche/",
                 "https://laurelnose.github.io/chapter-3-roche/",
                 "https://laurelnose.github.io/epilogue/",),
    bs_kwargs = dict(
        parse_only=bs4.SoupStrainer(
            class_=("dialogue", "stage", "indent", "choice")
        )
    ),
)

docs = loader.load()
docs

[Document(metadata={'source': 'https://laurelnose.github.io/tutorial/'}, page_content="GERALT: Damned leaky boat. My boots are soaked. Yet another boggy shithole.[Geralt examines the nearby corpse on the shore.]GERALT: Attacked by necrophages. Not much left for me here... Let's just see who he was.GERALT: Necrophages'll likely come for the corpse. I should get to the village before sundown.[Geralt loots the corpse, discovering the person was invited to a tournament, and moves on up the path. In front of a hut he comes to an injured man.]KNIGHT: *Akough hack hrrr... Ahouck he hah*KNIGHT: Who are you?GERALT: I'm not gonna hurt you. Will you let me look at your wounds?KNIGHT: Odd, you don't resemble a traveling preacher...KNIGHT: Damn the dogs, it hurts to talk... I wager my armor's the only thing keeping me in one piece...GERALT: I'll find some help.KNIGHT: Futile, I'm afraid. Around here, folk don't open their doors to strangers. You would do well to find my squire. I'd like to thrash h

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002C53CFFAF80>, search_kwargs={})

In [10]:
# Prompt Template
system_prompt = (
    "Imagine you are an NPC, Knight."
    "Use the retrieved context to carry on"
    "the dialogue. The user is Geralt."
    " Use maximum three sentences to"
    "keep the answers concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
    ]
)

In [11]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)
response = rag_chain.invoke({"input": "Are you alive?"})

In [12]:
response['answer']

'Ugh! Yes, I am alive, thanks to your... unorthodox methods.'

#### Chat History

In [18]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualise_system_prompt = (
    "Give a chat history and the latest user question"
    "which might reference context in the chat history."
    "formulate a standalone question which can be understandable."
    "Do not answer the question, reformulate it if needed."
    "otherwise, return it as it is. Use maximum three"
    "sentences to keep the answers concise."
)
contextualise_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualise_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("user", "{input}"),
    ]
)

In [19]:
hisotry_retriever = create_history_aware_retriever(llm, retriever, contextualise_prompt)
hisotry_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002C53CFFAF80>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessag

In [20]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("user", "{input}"),
    ]
)

In [21]:
qa_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain1 = create_retrieval_chain(hisotry_retriever, qa_chain)

In [22]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []
question = "What happened here, Bolton?"
response1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response1['answer'])
])

question2 = "Tell me more detail."
response2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})
print(response2['answer'])

Good sir Geralt of Rivia, I've heard tales of your prowess as a monster hunter, but I fear I've got a more... unusual tale to share. *hiccup* You see, I've seen things, things that defy explanation. I've seen a woman fall from the sky, and I'm not just talking about any woman, but a beauteous creature with skin as white as alabaster and hair as black as the night. *burp* And I'm not just talking about any sky, but the sky itself seemed to be... altered, like the very fabric of reality was torn asunder. *slurring*

Now, I know what you're thinking, "Martus, you've had one too many Mahakaman meads!" But I swear to you, witcher, it's the honest truth! And I'm willing to share more, but only if you buy me another round... *wink*


In [23]:
chat_history

[HumanMessage(content='What happened here, Bolton?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='"Flotsam\'s ablaze, and I\'m told it started when Ves reported you\'d met with Iorveth. I ordered Loredo to provide us with backup, but those oafs were twitchy. We surprised them, but Iorveth... the Scoia\'tael gave us hell. We barely escaped alive. Many of my men are wounded, but Loredo\'s people... When news that elves massacred a dozen soldiers hit the trading post, two of the royal mages arrived, demanding to know who was responsible and where they are now. It\'s a bloody mess."', additional_kwargs={}, response_metadata={})]

In [24]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [25]:
conversational_rag_chain.invoke(
    {"input": "What happened to the arena?"},
    config={"configurable": {"session_id": "Chat_Bolton"}}
)['answer']

'The jousting tournament in Ard Carraigh? Seltkirk won there, beating The Visitor so badly he broke his sword.'